## 사진 별 폴더 생성으로 수정

In [ ]:
from bs4 import BeautifulSoup
from selenium import webdriver
import urllib.request
import urllib
import time
import sys
import math
import os

#크롤링 요청 및 정보
print("=" * 50)
print("대한민국 구석구석 사이트의 여행지 정보 이미지 수집하기")
print("-" * 50)

query_txt = input("1. 크롤링할 키워드는 무엇입니까? : ")
cnt = int(input("2. 크롤링할 건수는 몇 건입니까? : "))
page_cnt = math.ceil(cnt/10)
print("출력될 페이지 수 : ", page_cnt)
f_dir = input("3. 사진을 저장할 폴더를 지정하세요. (예: C:\\LHR\\python\\notebook\\data\\) : ")
print("=" * 50)

#현재시간
n = time.localtime() 
s = "{}-{}-{}-{}-{}-{}".format(n.tm_year, n.tm_mon, n.tm_mday, n.tm_hour, n.tm_min, n.tm_sec)

dir_name = f_dir + s +'-' + query_txt
os.makedirs(dir_name) #data에 새로운 디렉토리 생성
dir_name += "\\"

s_time = time.time() #시작 타임스탬프


#창 열기
path = "C:/LHR/python/datadown/chromedriver.exe"
driver = webdriver.Chrome(path)
driver.get('https://korean.visitkorea.or.kr')
time.sleep(2)

driver.maximize_window() #화면 크기를 크게 한다


#코로나창
try :
    driver.find_element_by_xpath('//*[@id="chkForm01"]').click()
except :
    print("코로나 창이 없습니다")

#검색 키워드 검색창에 입력 -> 서치
element = driver.find_element_by_id("inp_search")
element.send_keys(query_txt)
element.send_keys("\n")

In [3]:
#페이지 관련
no = 1          #페이지 시작 번호      
catalog = {}
    
for x in range(1, page_cnt + 1) :
    print("{}페이지의 이미지 수집을 시작합니다 ===================".format(x))
        
    #광고 때문에 혹시 모르니 12로 설정
    for i in range(1, 12):
        if no > cnt:
            break
        
        #각 게시글의 제목 클릭
        try:         
            driver.find_element_by_xpath('''//*[@id="listBody"]/ul/li[{}]/div[2]/div[1]/a'''.format(i)).click()
        except:
            continue
        
        time.sleep(2)
        
        
        #상세페이지에 있는 값 읽어오기
        html = driver.page_source 
        soup = BeautifulSoup(html, 'html.parser')
            
        #이미지 주소 추출
        img_src2 = [] #img 리스트

        
        try:
            try:
                #유형 1. 보통게시글
                #폴더 생성
                title = soup.find('div', id='contents').find('h2').get_text()

                img_src = soup.find('div', 'box_txtPhoto').find_all('img')

                #보통 게시글 이미지 불러오기
                for i in img_src:
                    img_src = i['src']
                    img_src2.append(img_src)

                catalog[title]=img_src2
                print('[',no,']',' 유형1:',title)
            
            except:
                try:
                    #유형 2. 카드 게시글
                    #폴더 생성
                    title = soup.find('div', id='contents').find('h2').get_text()

                    contents_num = 4

                    for j in range(1, contents_num + 1):
                        driver.find_element_by_xpath('//*[@id="contents"]/div[2]/div[6]/div[1]/div[1]/div/ul/li[{}]/a'.format(j)).click()
                        time.sleep(2)

                        html = driver.page_source
                        soup = BeautifulSoup(html,'html.parser')

                        tab = soup.find('div',"cos_cont active")
                        tab2 = tab.find('div','info_area pc').find_all('a')

                        for i in tab2:
                            img_src = i['style'].split("(")[1].split(")")[0]
                            img_src2.append(img_src) #리스트에 저장

                    catalog[title]=img_src2
                    print('[',no,']', ' 유형2:',title)

                except:
                    try:
                        #상세페이지 제목받아오기 (폴더이름으로 사용예정)
                        #title = soup.find('div','titTypeWrap').find('h3').get_text()
                        #유형3에서 title이 div titTypeWrap이 아닌경우가 있음.
                        title = soup.find('div','titleType1').find('h2',id='topTitle').get_text()

                        #상세페이지에서 이미지 리스트받아오기
                        img_group = soup.find('div','swiper-wrapper').find_all(['div','style']) #pImgList를 담은 div태그

                        for k in img_group:
                            img_src = (str(k).split('("')[1].split('")')[0])
                            #url 보정 ('amp'빼야함)
                            img_src = img_src.replace('amp;','')
                            img_src2.append(img_src)

                        catalog[title]= img_src2
                        print(no,') 유형3:',title)
                        
                    except:
                        print(no,') 유형unkown: 이미지를 가져올 수 없습니다.')

#                     driver.find_element_by_xpath('''//*[@id="pImgList"]/div[1]/a''').click()
#                     time.sleep(2)

#                     for k in range(1, 11):
#                         html = driver.page_source
#                         soup = BeautifulSoup(html,'html.parser')

#                         img_group = soup.find('div', 'wrap').find_all('img')

#                         for f in img_group:
#                             img_src = (str(k).split('("')[1].split('")')[0])
#                             img_src2.append(img_src)

#                         driver.find_element_by_xpath('''//*[@id="detailPop"]/div[1]/div/div[1]/div/div[2]''').click()
#                         time.sleep(2)

#                     catalog[title]=img_src2
#                     print('[',no,']',' 유형3:',title)

        except:
            no +=1
            driver.back()
            continue
            
        no +=1                 
        driver.back()
        time.sleep(2)
        
    x += 1
    print('page','/',x)             
    if(x % 5 == 1):
        driver.find_element_by_link_text("다음").click()
    else:
        driver.find_element_by_link_text("{}".format(x)).click()
        
    time.sleep(2)  
    #여기까지 page별 loop                  

1페이지의 이미지 수집을 시작합니다 ===================
[ 1 ]  유형1: [강릉] 정동진 근처 가성비 좋은 숙소, 카리브모텔
page / 2


NoSuchElementException: Message: no such element: Unable to locate element: {"method":"link text","selector":"2"}
  (Session info: chrome=89.0.4389.128)


In [ ]:
f_dir_list=[] #폴더 리스트
title_list = list(catalog.keys())
img_list = list(catalog.values())

print(title_list)
print(img_list)


#title별 폴더만들기
for t in title_list:
    f_dir_name = dir_name+s+'-'+t
    os.makedirs(f_dir_name)
    
    #폴더 리스트에 추가
    f_dir_list.append(f_dir_name)

    
#폴더별로 이미지 저장  
for i in range(0,len(img_list)):
    
    file_no=0 #이미지 순번   
    
    #이미지를 저장하려는 위치로 디렉토리를 변경
    os.chdir(f_dir_list[i])
    print("폴더: "+f_dir_list[i]+"에 저장 시작:")    
    
    for j in range(0,len(img_list[i])):
        try:
            urllib.request.urlretrieve(img_list[i][j],str(file_no)+'.jpg')
        except:
            print(e)
            continue
        file_no+=1
        time.sleep(0.5)
        print("{}번째 이미지 저장중입니다.".format(file_no))

In [ ]:
#이미지 크롤링 후 요약정보 출력
e_time = time.time()
d_time = e_time - s_time

print("=" *80)
print("총 소요시간은 %s 초 입니다 " %round(d_time,1))
total=0
for i in img_list:
    total += len(i)
print("총 저장 건수는 %s 건 입니다 " %(total))
print("파일이 저장된 폴더들의 경로는 입니다")
for x in f_dir_list:
    print(x)
print("=" *80)
 
driver.close( )